<a href="https://colab.research.google.com/github/chw8207/pytorch_study/blob/main/%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98_%EA%B8%B0%EC%B4%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
from torch import optim, nn
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0' :
    raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

Found GPU at: /device:GPU:0


In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16894799595682062605
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15510929408
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13986138932465722896
physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0"
xla_global_id: 416903419
]


In [4]:
print(torch.__version__)
print(torch.cuda.is_available())

2.1.0+cu121
True


### 텐서 생성

In [ ]:
print(torch.tensor([1,2,3]))
print(torch.Tensor([[1,2,3],[4,5,6]]))
print(torch.LongTensor([1,2,3]))
print(torch.FloatTensor([1,2,3]))

tensor([1, 2, 3])
tensor([[1., 2., 3.],
        [4., 5., 6.]])
tensor([1, 2, 3])
tensor([1., 2., 3.])


### 텐서 속성

In [ ]:
tensor = torch.rand(1,2)
print(tensor)
print(tensor.shape)
print(tensor.dtype)
print(tensor.device)

tensor([[0.4146, 0.6093]])
torch.Size([1, 2])
torch.float32
cpu


### 차원변환

In [ ]:
tensor = torch.rand(1,2)
print(tensor)
print(tensor.shape)

tensor([[0.8151, 0.4838]])
torch.Size([1, 2])


In [ ]:
tensor = tensor.reshape(2,1)
print(tensor)
print(tensor.shape)

tensor([[0.8151],
        [0.4838]])
torch.Size([2, 1])


### 자료형 설정

In [ ]:
tensor = torch.rand((3,3), dtype=torch.float)
print(tensor)

tensor([[0.5237, 0.1550, 0.4514],
        [0.1769, 0.7696, 0.8515],
        [0.5145, 0.7181, 0.3754]])


### 장치 설정

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cpu = torch.FloatTensor([1,2,3])
gpu = torch.cuda.FloatTensor([1,2,3])
tensor = torch.rand((1,1), device=device)
print(device)
print(cpu)
print(gpu)
print(tensor)

<ipython-input-4-b0dbfd40ce26>:3: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  gpu = torch.cuda.FloatTensor([1,2,3])


cuda
tensor([1., 2., 3.])
tensor([1., 2., 3.], device='cuda:0')
tensor([[0.3150]], device='cuda:0')


### 장치 변환

In [ ]:
cpu = torch.FloatTensor([1,2,3])
gpu = cpu.cuda()
gpu2cpu = gpu.cpu()
cpu2gpu = cpu.to('cuda')
print(cpu)
print(gpu)
print(gpu2cpu)
print(cpu2gpu)

tensor([1., 2., 3.])
tensor([1., 2., 3.], device='cuda:0')
tensor([1., 2., 3.])
tensor([1., 2., 3.], device='cuda:0')


### 넘파이 배열의 텐서 변환

In [ ]:
ndarray = np.array([1,2,3], dtype=np.uint8)
print(torch.tensor(ndarray))
print(torch.Tensor(ndarray))
print(torch.from_numpy(ndarray))

tensor([1, 2, 3], dtype=torch.uint8)
tensor([1., 2., 3.])
tensor([1, 2, 3], dtype=torch.uint8)


### 텐서의 넘파이 배열 변환

In [ ]:
tensor = torch.cuda.FloatTensor([1,2,3])
ndarray = tensor.detach().cpu().numpy()
print(ndarray)
print(type(ndarray))

[1. 2. 3.]
<class 'numpy.ndarray'>


### 단순선형회귀: 넘파이

In [ ]:
X = np.array(
    [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10],
    [11], [12], [13], [14], [15], [16], [17], [18], [19], [20],
    [21], [22], [23], [24], [25], [26], [27], [28], [29], [30]]
)
y = np.array(
    [[0.94], [1.98], [2.88], [3.92], [3.96], [4.55], [5.64], [6.3], [7.44], [9.1],
    [8.46], [9.5], [10.67], [11.16], [14], [11.83], [14.4], [14.25], [16.2], [16.32],
    [17.46], [19.8], [18], [21.34], [22], [22.5], [24.57], [26.04], [21.6], [28.8]]
)

In [ ]:
# 하이퍼파라미터 초기화
weight = 0.0
bias = 0.0
learning_rate = 0.005

In [ ]:
# 에폭 설정
for epoch in range(10000) :
  # 손실 함수 및 가설 선언
  y_hat = weight*X + bias
  cost = ((y-y_hat)**2).mean()

  # 가중치와 편향 갱신
  weight = weight - learning_rate*((y_hat-y)*X).mean()
  bias = bias - learning_rate*(y_hat-y).mean()

  if (epoch + 1) % 1000 == 0 :
    print(f'Epoch: {epoch+1:4d}, Weight: {weight:.3f}, Bias: {bias:.3f}, Cost: {cost:.3f}')

Epoch: 1000, Weight: 0.872, Bias: -0.290, Cost: 1.377
Epoch: 2000, Weight: 0.877, Bias: -0.391, Cost: 1.373
Epoch: 3000, Weight: 0.878, Bias: -0.422, Cost: 1.372
Epoch: 4000, Weight: 0.879, Bias: -0.432, Cost: 1.372
Epoch: 5000, Weight: 0.879, Bias: -0.435, Cost: 1.372
Epoch: 6000, Weight: 0.879, Bias: -0.436, Cost: 1.372
Epoch: 7000, Weight: 0.879, Bias: -0.436, Cost: 1.372
Epoch: 8000, Weight: 0.879, Bias: -0.436, Cost: 1.372
Epoch: 9000, Weight: 0.879, Bias: -0.436, Cost: 1.372
Epoch: 10000, Weight: 0.879, Bias: -0.436, Cost: 1.372


- 초깃값 다르게: learning_rate=0.001

In [ ]:
# 하이퍼파라미터 초기화
weight = 0.0
bias = 0.0
learning_rate = 0.001

In [ ]:
# 에폭 설정
for epoch in range(10000) :
  # 손실 함수 및 가설 선언
  y_hat = weight*X + bias
  cost = ((y-y_hat)**2).mean()

  # 가중치와 편향 갱신
  weight = weight - learning_rate*((y_hat-y)*X).mean()
  bias = bias - learning_rate*(y_hat-y).mean()

  if (epoch + 1) % 1000 == 0 :
    print(f'Epoch: {epoch+1:4d}, Weight: {weight:.3f}, Bias: {bias:.3f}, Cost: {cost:.3f}')

Epoch: 1000, Weight: 0.860, Bias: -0.059, Cost: 1.406
Epoch: 2000, Weight: 0.864, Bias: -0.138, Cost: 1.393
Epoch: 3000, Weight: 0.867, Bias: -0.201, Cost: 1.385
Epoch: 4000, Weight: 0.870, Bias: -0.251, Cost: 1.380
Epoch: 5000, Weight: 0.872, Bias: -0.290, Cost: 1.377
Epoch: 6000, Weight: 0.873, Bias: -0.321, Cost: 1.375
Epoch: 7000, Weight: 0.874, Bias: -0.345, Cost: 1.374
Epoch: 8000, Weight: 0.875, Bias: -0.364, Cost: 1.373
Epoch: 9000, Weight: 0.876, Bias: -0.379, Cost: 1.373
Epoch: 10000, Weight: 0.877, Bias: -0.391, Cost: 1.373


- 초깃값 다르게: learning_rate=0.006

In [ ]:
# 하이퍼파라미터 초기화
weight = 0.0
bias = 0.0
learning_rate = 0.006

In [ ]:
# 에폭 설정
for epoch in range(10000) :
  # 손실 함수 및 가설 선언
  y_hat = weight*X + bias
  cost = ((y-y_hat)**2).mean()

  # 가중치와 편향 갱신
  weight = weight - learning_rate*((y_hat-y)*X).mean()
  bias = bias - learning_rate*(y_hat-y).mean()

  if (epoch + 1) % 1000 == 0 :
    print(f'Epoch: {epoch+1:4d}, Weight: {weight:.3f}, Bias: {bias:.3f}, Cost: {cost:.3f}')

Epoch: 1000, Weight: 0.873, Bias: -0.321, Cost: 1.375
Epoch: 2000, Weight: 0.878, Bias: -0.408, Cost: 1.372
Epoch: 3000, Weight: 0.879, Bias: -0.429, Cost: 1.372
Epoch: 4000, Weight: 0.879, Bias: -0.434, Cost: 1.372
Epoch: 5000, Weight: 0.879, Bias: -0.436, Cost: 1.372
Epoch: 6000, Weight: 0.879, Bias: -0.436, Cost: 1.372
Epoch: 7000, Weight: 0.879, Bias: -0.436, Cost: 1.372
Epoch: 8000, Weight: 0.879, Bias: -0.436, Cost: 1.372
Epoch: 9000, Weight: 0.879, Bias: -0.436, Cost: 1.372
Epoch: 10000, Weight: 0.879, Bias: -0.436, Cost: 1.372


### 단순선형회귀: 파이토치

In [ ]:
X = torch.FloatTensor(
    [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10],
    [11], [12], [13], [14], [15], [16], [17], [18], [19], [20],
    [21], [22], [23], [24], [25], [26], [27], [28], [29], [30]]
)
y = torch.FloatTensor(
    [[0.94], [1.98], [2.88], [3.92], [3.96], [4.55], [5.64], [6.3], [7.44], [9.1],
    [8.46], [9.5], [10.67], [11.16], [14], [11.83], [14.4], [14.25], [16.2], [16.32],
    [17.46], [19.8], [18], [21.34], [22], [22.5], [24.57], [26.04], [21.6], [28.8]]
)

In [ ]:
# 하이퍼파라미터 초기화
weight = torch.zeros(1, requires_grad=True)
bias = torch.zeros(1, requires_grad=True)
learning_rate = 0.001

In [ ]:
# 최적화 선언
optimizer = optim.SGD([weight,bias], lr=learning_rate)

In [ ]:
# 에폭, 가설, 손실 함수 선언
for epoch in range(10000) :
  hypothesis = X*weight + bias
  cost = torch.mean((hypothesis-y)**2)

  # 가중치와 편향 갱신
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if (epoch + 1) % 1000 == 0:
    print(f"Epoch : {epoch+1:4d}, Weight : {weight.item():.3f}, Bias : {bias.item():.3f}, Cost : {cost:.3f}")

Epoch : 1000, Weight : 0.873, Bias : -0.321, Cost : 1.375
Epoch : 2000, Weight : 0.875, Bias : -0.364, Cost : 1.373
Epoch : 3000, Weight : 0.877, Bias : -0.391, Cost : 1.373
Epoch : 4000, Weight : 0.878, Bias : -0.408, Cost : 1.372
Epoch : 5000, Weight : 0.878, Bias : -0.419, Cost : 1.372
Epoch : 6000, Weight : 0.878, Bias : -0.425, Cost : 1.372
Epoch : 7000, Weight : 0.879, Bias : -0.429, Cost : 1.372
Epoch : 8000, Weight : 0.879, Bias : -0.432, Cost : 1.372
Epoch : 9000, Weight : 0.879, Bias : -0.433, Cost : 1.372
Epoch : 10000, Weight : 0.879, Bias : -0.434, Cost : 1.372


In [ ]:
# 에폭, 가설, 손실 함수 선언
for epoch in range(10000) :
  hypothesis = X*weight + bias
  cost = torch.mean((hypothesis-y)**2)

  print(f'Epoch: {epoch+1:4d}')
  print(f'Step [1]: Gradient: {weight.grad}, Weight: {weight.item():.5f}')

  optimizer.zero_grad()
  print(f'Step [2]: Gradient: {weight.grad}, Weight: {weight.item():.5f}')

  cost.backward()
  print(f'Step [3]: Gradient: {weight.grad}, Weight: {weight.item():.5f}')

  optimizer.step()
  print(f'Step [4]: Gradient: {weight.grad}, Weight: {weight.item():.5f}')

  if epoch==3:
    break

Epoch:    1
Step [1]: Gradient: None, Weight: 0.00000
Step [2]: Gradient: None, Weight: 0.00000
Step [3]: Gradient: tensor([-540.4854]), Weight: 0.00000
Step [4]: Gradient: tensor([-540.4854]), Weight: 0.54049
Epoch:    2
Step [1]: Gradient: tensor([-540.4854]), Weight: 0.54049
Step [2]: Gradient: None, Weight: 0.54049
Step [3]: Gradient: tensor([-198.9818]), Weight: 0.54049
Step [4]: Gradient: tensor([-198.9818]), Weight: 0.73947
Epoch:    3
Step [1]: Gradient: tensor([-198.9818]), Weight: 0.73947
Step [2]: Gradient: None, Weight: 0.73947
Step [3]: Gradient: tensor([-73.2604]), Weight: 0.73947
Step [4]: Gradient: tensor([-73.2604]), Weight: 0.81273
Epoch:    4
Step [1]: Gradient: tensor([-73.2604]), Weight: 0.81273
Step [2]: Gradient: None, Weight: 0.81273
Step [3]: Gradient: tensor([-26.9772]), Weight: 0.81273
Step [4]: Gradient: tensor([-26.9772]), Weight: 0.83970


### 신경망 패키지

In [ ]:
X = torch.FloatTensor(
    [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10],
    [11], [12], [13], [14], [15], [16], [17], [18], [19], [20],
    [21], [22], [23], [24], [25], [26], [27], [28], [29], [30]]
)
y = torch.FloatTensor(
    [[0.94], [1.98], [2.88], [3.92], [3.96], [4.55], [5.64], [6.3], [7.44], [9.1],
    [8.46], [9.5], [10.67], [11.16], [14], [11.83], [14.4], [14.25], [16.2], [16.32],
    [17.46], [19.8], [18], [21.34], [22], [22.5], [24.57], [26.04], [21.6], [28.8]]
)

In [ ]:
# 모델 선언
model = nn.Linear(1,1, bias=True)
criterion = torch.nn.MSELoss()
learning_rate = 0.001

In [ ]:
# 순방향 연산
for epoch in range(10000) :
  output = model(X)
  cost = criterion(output, y)

  if (epoch+1) % 1000 == 0 :
    print(f'Epoch: {epoch+1:4d}, Model: {list(model.parameters())}, Cost: {cost:.3f}')

Epoch: 1000, Model: [Parameter containing:
tensor([[-0.7150]], requires_grad=True), Parameter containing:
tensor([0.8314], requires_grad=True)], Cost: 741.075
Epoch: 2000, Model: [Parameter containing:
tensor([[-0.7150]], requires_grad=True), Parameter containing:
tensor([0.8314], requires_grad=True)], Cost: 741.075
Epoch: 3000, Model: [Parameter containing:
tensor([[-0.7150]], requires_grad=True), Parameter containing:
tensor([0.8314], requires_grad=True)], Cost: 741.075
Epoch: 4000, Model: [Parameter containing:
tensor([[-0.7150]], requires_grad=True), Parameter containing:
tensor([0.8314], requires_grad=True)], Cost: 741.075
Epoch: 5000, Model: [Parameter containing:
tensor([[-0.7150]], requires_grad=True), Parameter containing:
tensor([0.8314], requires_grad=True)], Cost: 741.075
Epoch: 6000, Model: [Parameter containing:
tensor([[-0.7150]], requires_grad=True), Parameter containing:
tensor([0.8314], requires_grad=True)], Cost: 741.075
Epoch: 7000, Model: [Parameter containing:
ten

## 데이터세트와 데이터로더

### 다중선형회귀

In [6]:
# 학습 데이터 선언
train_X = torch.FloatTensor([
    [1,2],[2,3],[3,4],[4,5],[5,6],[6,7]
])
train_y = torch.FloatTensor([
    [0.1,1.5],[1,2.8],[1.9,4.1],[2.8,5.4],[3.7,6.7],[4.6,8]
])

In [8]:
# 데이터세트와 데이터 로더
train_dataset = TensorDataset(train_X, train_y)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, drop_last=True)

In [9]:
# 모델, 오차함수, 최적화 함수 선언
model = nn.Linear(2, 2, bias=True)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [10]:
# 데이터로더 적용
for epoch in range(20000):
  # 에폭마다 오차를 다시 계산하기 위해 초기화
  cost = 0

  for batch in train_dataloader:
    X, y = batch
    output = model(X)

    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    cost += loss

  cost = cost / len(train_dataloader)

  if (epoch + 1) % 1000 == 0 :
    print(f'Epoch: {epoch+1:4d}, Model: {list(model.parameters())}, Cost: {cost:.3f}')

Epoch: 1000, Model: [Parameter containing:
tensor([[0.2332, 0.4948],
        [0.5020, 0.7758]], requires_grad=True), Parameter containing:
tensor([-0.5722, -0.4823], requires_grad=True)], Cost: 0.051
Epoch: 2000, Model: [Parameter containing:
tensor([[0.2563, 0.4829],
        [0.5050, 0.7742]], requires_grad=True), Parameter containing:
tensor([-0.6073, -0.4868], requires_grad=True)], Cost: 0.045
Epoch: 3000, Model: [Parameter containing:
tensor([[0.2779, 0.4718],
        [0.5078, 0.7728]], requires_grad=True), Parameter containing:
tensor([-0.6400, -0.4910], requires_grad=True)], Cost: 0.039
Epoch: 4000, Model: [Parameter containing:
tensor([[0.2981, 0.4613],
        [0.5104, 0.7714]], requires_grad=True), Parameter containing:
tensor([-0.6706, -0.4950], requires_grad=True)], Cost: 0.034
Epoch: 5000, Model: [Parameter containing:
tensor([[0.3170, 0.4516],
        [0.5128, 0.7702]], requires_grad=True), Parameter containing:
tensor([-0.6992, -0.4987], requires_grad=True)], Cost: 0.030


- 편향 제거

In [11]:
# 모델, 오차함수, 최적화 함수 선언
model = nn.Linear(2, 2, bias=False)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [12]:
# 데이터로더 적용
for epoch in range(20000):
  # 에폭마다 오차를 다시 계산하기 위해 초기화
  cost = 0

  for batch in train_dataloader:
    X, y = batch
    output = model(X)

    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    cost += loss

  cost = cost / len(train_dataloader)

  if (epoch + 1) % 1000 == 0 :
    print(f'Epoch: {epoch+1:4d}, Model: {list(model.parameters())}, Cost: {cost:.3f}')

Epoch: 1000, Model: [Parameter containing:
tensor([[0.3124, 0.3218],
        [0.3771, 0.7844]], requires_grad=True)], Cost: 0.154
Epoch: 2000, Model: [Parameter containing:
tensor([[0.3438, 0.2964],
        [0.3935, 0.7712]], requires_grad=True)], Cost: 0.148
Epoch: 3000, Model: [Parameter containing:
tensor([[0.3744, 0.2716],
        [0.4095, 0.7582]], requires_grad=True)], Cost: 0.141
Epoch: 4000, Model: [Parameter containing:
tensor([[0.4044, 0.2474],
        [0.4251, 0.7456]], requires_grad=True)], Cost: 0.135
Epoch: 5000, Model: [Parameter containing:
tensor([[0.4337, 0.2237],
        [0.4404, 0.7333]], requires_grad=True)], Cost: 0.129
Epoch: 6000, Model: [Parameter containing:
tensor([[0.4624, 0.2005],
        [0.4553, 0.7212]], requires_grad=True)], Cost: 0.123
Epoch: 7000, Model: [Parameter containing:
tensor([[0.4904, 0.1779],
        [0.4699, 0.7094]], requires_grad=True)], Cost: 0.117
Epoch: 8000, Model: [Parameter containing:
tensor([[0.5177, 0.1558],
        [0.4841, 0.69

## 모델/데이터세트 분리

### 비선형 회귀

In [14]:
# 사용자 정의 데이터세트
class CustomDataset(Dataset):
  def __init__(self, file_path):
    df = pd.read_csv(file_path)
    self.x = df.iloc[:,0].values
    self.y = df.ilco[:,1].values
    self.length = len(df)

  def __getitem__(self, index):
    x = torch.FloatTensor([self.x[index]**2, self.x[index]])
    y = torch.FloatTensor([self.y[index]])

  def __len__(self):
    return self.length

In [15]:
# 사용자 정의 모델
class CustomModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer = nn.Linear(2,1)

  def forward(self, x):
    x = self.layer(x)
    return x

In [16]:
# 사용자 정의 데이터세트와 데이터로더
train_datset = CustomDataset("")

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/non_linear.csv'